In [ ]:
using LinearAlgebra, Plots, LaTeXStrings # Necessary packages
# --- Run in Google Colab if Julia is not locally installed --- #

In [ ]:
function build_ssh_hamiltonian(N_cells::Int, t1::Float64, t2::Float64) # Build the SSH Hamiltonian for a chain with N_cells unit cells
    Nsites = 2 * N_cells
    H = zeros(Float64, Nsites, Nsites)
    # intracell hopping t1 between A_n (2n-1) and B_n (2n)
    # intercell hopping t2 between B_n (2n) and A_{n+1} (2n+1)
    for n in 1:N_cells
        a = 2n - 1
        b = 2n
        H[a, b] = -t1
        H[b, a] = -t1
        if n < N_cells
            H[b, 2n + 1] = -t2
            H[2n + 1, b] = -t2
        end
    end
    return H
end

build_ssh_hamiltonian (generic function with 1 method)

In [ ]:
# -------------------------
# Diagonalize and utilities
# -------------------------
function spectrum_and_states(H::AbstractMatrix{T}) where T # Diagonalize Hamiltonian H
    vals, vecs = eigen(H)  # vals sorted in ascending order
    return vals, vecs
end

spectrum_and_states (generic function with 1 method)

In [ ]:
# Detect edge-localized states near zero energy
function detect_edge_states(vals::AbstractVector{T}, vecs::AbstractMatrix{T}; #Find edge states based on wavefunction localization
                            energy_tol::Float64 = 1e-2,
                            edge_weight_thresh::Float64 = 0.2,
                            edge_size_sites::Int = 2,
                            report_energies::Bool = false) where T
    # Find eigenstates with a large fraction of probability on the
    # first/last `edge_size_sites` sites. This is more robust for finite systems
    # where zero-mode energies are not exactly zero.
    Nsites = size(vecs, 1) #Declare number of sites from eigenvector matrix
    edge_indices = Int[] #Initialize array to hold indices of edge states
    edge_energies = Float64[] #Initialize array to hold energies of edge states

    for i in 1:length(vals) #Loop over all eigenvalues
        psi = vecs[:, i]
        p_left = sum(abs2, psi[1: min(edge_size_sites, Nsites)])
        p_right = sum(abs2, psi[max(1, Nsites-edge_size_sites+1):Nsites])
        if max(p_left, p_right) >= edge_weight_thresh
            push!(edge_indices, i)
            push!(edge_energies, vals[i])
        end
    end

    if report_energies && !isempty(edge_energies) #Report detected edge-state energies if requested
        println("Detected edge-state energies: ", round.(edge_energies, sigdigits=6))
    end
    return unique(edge_indices), length(edge_indices)
end

detect_edge_states (generic function with 1 method)

In [ ]:
# -------------------------
# Sweep and plotting
# -------------------------
function sweep_t2_and_plot(Ncells::Int, t1::Float64; t2_vals = range(0.0, stop=2.0, length=201), # Function to sweep t2 and plot results
                          energy_tol = 1e-3)
    # Inialize lists for storing all energies and corresponding t_2/t1
    all_energies = Float64[]
    all_t2_over_t1 = Float64[]
    edge_counts = Int[]
    
    for t2 in t2_vals # Loop over t2 values
        H = build_ssh_hamiltonian(Ncells, t1, t2) # Build Hamiltonian
        vals, vecs = spectrum_and_states(H) # Diagonalize
        
        # Store all energies and corresponding t2/t1
        append!(all_energies, vals)
        append!(all_t2_over_t1, fill(t2 / t1, length(vals)))

        # Compute edge states
        _, cnt = detect_edge_states(vals, vecs; energy_tol=energy_tol)
        push!(edge_counts, cnt)
    end
    
    # Plot full spectrum vs t2/t1
    plt = scatter(all_t2_over_t1, all_energies, 
                  xlabel = L"t_2/t_1", ylabel = L"E", 
                  title = L"SSH Energies vs $t_2/t_1$ ($t_1$=%$(t1), Num. Cells=%$(Ncells))", 
                  legend=false, markersize=1, markerstrokewidth=0, 
                  color=:blue, alpha=0.5)
    savefig(plt, "ssh_full_spectrum_vs_t2.png")
    
    # Plot edge count vs t2/t1
    # Plot edge count vs t2/t1 (Just for t2 > 0)
    mask = t2_vals .> 0  # Boolean mask for t2 > 0
    plt2 = plot((t2_vals ./ t1)[mask], edge_counts[mask], seriestype=:steppre, xlabel=L"t_2/t_1", ylabel="# edge states",
                title = L"Num. of Edge States vs $t_2/t_1$ ($t_2$ > 0)", ylim = (-0.2, 4.2), legend=false)
    savefig(plt2, "ssh_edgecount_vs_t2.png")
    
    return t2_vals, all_energies, edge_counts
end

sweep_t2_and_plot (generic function with 1 method)

In [51]:
# -------------------------
# Main demo runner
# -------------------------
function main()
    println("SSH OBC demo: construyendo y analizando espectros...")
    Ncells = 20
    t1 = 1.0
    t2_vals = range(0.0, stop=2.0, length=301)
    t2_vals, all_energies, edge_counts = sweep_t2_and_plot(Ncells, t1; t2_vals=t2_vals, energy_tol=1e-3)
    println("Guardadas figuras: ssh_full_spectrum_vs_t2.png y ssh_edgecount_vs_t2.png")

end


main (generic function with 1 method)

In [52]:
main()

SSH OBC demo: construyendo y analizando espectros...
Guardadas figuras: ssh_full_spectrum_vs_t2.png y ssh_edgecount_vs_t2.png
